# Bringing it all together

In this notebook we aim to apply several different stratifications, mixing and vaccination concepts learnt previously into a single model. The goal here is to take everything that we have learnt previously using "toy" code with simplified stratifications/functionality loosely representative of Covid transmission - and extend this to a model that we can be satisfied reasonably replicates most of the most important dynamics of Covid transmission in Malaysia.

A little like notebook number 04, this is an opportunity to take stock and apply the epidemiological principles we have worked through in the preceding notebooks.

## Standard preliminaries
Before we get into building the model, let's start off with some of our standard (or "boilerplate") code to get everything set up.

In [ ]:
# Get the mixing matrix for Malaysia
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/MYS_matrices.pkl

mixing_matrix = pd.read_pickle("MYS_matrices.pkl", compression='infer')
age_mixing_matrix = mixing_matrix["all_locations"]

In [ ]:
# pip install the required packages if running in Colab
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Standard imports, plotting option and constant definition
from datetime import datetime, timedelta
from typing import List, Union
import pandas as pd
import plotly.express as px
import numpy as np

from summer import CompartmentalModel
from summer import Stratification

pd.options.plotting.backend = "plotly"

COVID_BASE_DATE = datetime(2019, 12, 31)
region = "Malaysia"

In [ ]:
# Get a function to access the Malaysia data if running in Colab
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/get_mys_data.py

import get_mys_data

In [ ]:
from autumn.core.inputs.covid_mys.queries import get_mys_vac_coverage

In [ ]:
vaccination_data = pd.DataFrame(
    {
        "full": get_mys_vac_coverage(dose="full"),
        "booster": get_mys_vac_coverage(dose="booster"),
    }
)
vaccination_data.plot()

In [ ]:
# ... and use it to get the actual data
df = get_mys_data.fetch_mys_data()
initial_population = get_mys_data.get_initial_population(region)
observations = get_mys_data.get_target_observations(df, region, "cases")

In [ ]:
# Define the model running period and convert to a numeric representation
start_date = datetime(2021, 1, 30)
end_date = start_date + timedelta(days=300)
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations[start_date: end_date]["cases_new"]

# Model

## Define a model

In [ ]:
def build_unstratified_model(parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Flow parameters
    Returns:
        A compartmental model currently without stratification applied
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=["S", "E", "I", "R"],
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(
        distribution=
        {
            "S": initial_population - infectious_seed, 
            "I": infectious_seed
        }
    )
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")

    return model

In [ ]:
general_params = {
    "contact_rate": 0.1,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.002,
    "reporting_fraction": 0.09,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 200.,
}

model = build_unstratified_model(parameters=general_params)
model.ru

# Age stratification

Next, let's try a simple stratification where we split the population into 'young' (say, 0 to 18 years old) and 'old' (age 19 and above). 

In [ ]:
from summer import Stratification

def get_age_stratification(
    compartments_to_stratify: List[str], 
    age_params: dict,
    matrix: Union[np.ndarray, callable]
) -> Stratification:
    """
    Create a summer stratification object that stratifies compartments into
    strata, which are intended to represent age stratifications.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify
        age_params: A dictionary which speicifies age-related parameters to implement
        matrix: The mixing matrix we are applying for the age structure
    Returns:
        A summer stratification object to represent age stratification (not yet applied)
    """
    strata = ["young", "old"]
    
    if isinstance(matrix, np.ndarray):
        msg = "Mixing matrix is not 2-dimensional"
        assert matrix.ndim == 2, msg

        msg = f"Dimensions of the mixing matrix incorrect: {matrix.shape[0]}, {matrix.shape[1]}, {len(strata)}"
        assert matrix.shape[0] == matrix.shape[1] == len(strata), msg
        
    
     # Create the stratification
    age_strat = Stratification(name="age", strata=strata, compartments=["S","E", "I", "R"])

    # Create our population split dictionary, whose keys match the strata
    pop_split = {"young": 0.25, "old": 0.75}

    # Set a population distribution
    age_strat.set_population_split(pop_split)
    
    # Add an adjustment to the 'infection' flow
    age_strat.set_flow_adjustments(
        "infection",
        {
            "old": None,  
            "young":age_params["infection_level"] ,  
        },
    )

    # Add an adjustment to the 'infection_death' flow
    age_strat.set_flow_adjustments(
        "infection_death",
        {
            "old": age_params["severity_old"],  
            "young": age_params["severity_young"],  
        },
    )

    # Add an adjustment to the 'recovery' flow
    age_strat.set_flow_adjustments(
        "recovery",
        {
            "old": None,  # No adjustment for old people, use baseline
            "young": age_params["recovery_level"],  
        },
    )
    
    # Add the mixing matrix to the stratification
    age_strat.set_mixing_matrix(matrix)
    
    return age_strat

# Strain stratification

We aim to capture two different viral strains with our model: the wild-type strain and a newly emerging variant.

The new variant may have a different level of transmissibility and could be associated with a different risk of death, compared to the ancestral strain. We want to capture these differences with our model using a strain stratification.

In [ ]:
from summer import StrainStratification
def get_strain_stratification(
    compartments_to_stratify: List[str], 
    voc_params: dict
) -> Stratification:
    """
    Create a summer stratification object that stratifies compartments into
    strata, which are intended to represent infectious disease strains.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify
        voc_params: A dictionary which speicifies the infectiousness and severity of strains
    Returns:
        A summer stratification object to represent strain stratification (not yet applied)
    """
    strata = ["wild", "variant"]
    strat = StrainStratification(name="strain", strata=strata, compartments=compartments_to_stratify)

    # At the start of the simulation, a certain proportion of infected people have the variant strain.
    strat.set_population_split({'wild': 1. - voc_params["variant_start_prop"], 'variant': voc_params["variant_start_prop"]})

    # Adjusting the death risk associated with the new variant
    strat.set_flow_adjustments("infection_death", {
        "wild": None,
        "variant": voc_params["variant_rel_ifr"],
    })

    # Adjusting the transmissibility of the new variant.
    strat.set_flow_adjustments("infection", {
        "wild": None,
        "variant": voc_params["variant_rel_transmissibility"],
    })


    # Adjusting the progression rate of the new variant.
    strat.set_flow_adjustments("progression", {
        "wild": None,
        "variant": voc_params["variant_rel_progression"],
    })
    
    return strat

In [ ]:
def build_stratified_model(
    parameters: dict,
    compartments_to_stratify:List[str],
    age_params: dict,
    age_mixing_matrix: np.ndarray,
    voc_params: dict
) -> CompartmentalModel:
    """
    Get the model object with the vaccination stratification applied to it.
    
    Arguments:
        parameters: A dictionary containing the parameter values to use
        compartments_to_stratify: A list of compartments to which the stratification is applied
        age_params: A dictionary which speicifies the age specific parameters 
        voc_params: A dictionary which speicifies the infectiousness and lethality of the variant
    Returns:
        The model object        
    """
    
    # Get an unstratified model object
    model = build_unstratified_model(parameters)

    # Get and apply the age stratification
    age_strat = get_age_stratification(compartments_to_stratify, age_params, age_mixing_matrix)
    model.stratify_with(age_strat)

    # Get and apply the VoC stratification
    strain_strat = get_strain_stratification(compartments_to_stratify, voc_params)
    model.stratify_with(strain_strat)
    
    return model

In [ ]:
# This time, we'll create a function right away

def get_severity_strat() -> Stratification:
    # Create a stratification named 'severity', applying to the infectious, which
    # splits that compartment into 'asymptomatic', 'mild' and 'severe'.
    severity_strata = ["asymptomatic", "mild", "severe"]

    # Notice the new argument ["I"] for the compartment parameter.
    severity_strat = Stratification(name="severity", strata=severity_strata, compartments=["I"])

    # Set a population distribution - everyone starts out asymptomatic.
    severity_strat.set_population_split({"asymptomatic": 1.0, "mild": 0, "severe": 0})
    
    return severity_strat

# We need to call the function so we have a Stratification object to work with
severity_strat = get_severity_strat()

# Add an adjustment to the 'infection' flow, overriding default split.
severity_strat.set_flow_adjustments(
    "progression",
    {
        "asymptomatic": 0.3,  # 30% of incident cases are asymptomatic
        "mild": 0.5,  # 50% of incident cases are mild
        "severe": 0.2,  # 20% of incident cases are severe
    },
)

# Add an adjustment to the 'infection_death' flow
severity_strat.set_flow_adjustments(
    "infection_death",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)

severity_strat.add_infectiousness_adjustments(
    "I",
    {
        "asymptomatic": 0.5,
        "mild": None,
        "severe": 1.5,
    },
)

# Build and run model with the stratification we just defined
model = build_model_with_flows(parameters)
model.stratify_with(severity_strat)
model.run()

# Build and run the model

In [ ]:
# Build and run the stratified model with some fairly arbitrary parameters calibrated to the local epidemic
general_params = {
    "contact_rate": 0.1,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.002,
    "reporting_fraction": 0.09,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 200.,
}

age_params = {
    "infection_level": 2.0,
    "severity_old": 3.0,
    "severity_young": 0.5,
    "recovery_level": 0.5
}

voc_params = {
    "variant_start_prop": .1,
    "variant_rel_ifr": 1.2,
    "variant_rel_transmissibility": 2.,
    "variant_rel_progression": .5
}

compartments_to_stratify = ["S", "E", "I", "R"]
strat_model = build_stratified_model(general_params, compartments_to_stratify, age_params, age_matrix, voc_params)



In [ ]:
strat_model.run()

notifications = strat_model.get_derived_outputs_df()["notifications"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "modelled": notifications
    }
).plot()